# **SIR Agent-Based Model with Behavioral Heterogeneity**

This notebook demonstrates how to implement an agent-based SIR (Susceptible–Infected–Recovered) model using the **`sagesim`** library.

The classical **SIR model** provides a foundational framework in epidemiology for modeling the spread of infectious diseases. By extending it to an **agent-based model (ABM)**, we capture individual-level variation and interaction patterns, allowing us to simulate more realistic and heterogeneous dynamics of disease transmission.

In this implementation:

- Each **agent** represents an individual with a unique behavioral profile that influences their susceptibility to infection.
- Agents occupy one of three states:
  - **Susceptible**: Healthy, but at risk of infection.
  - **Infected**: Currently infected and capable of spreading the disease.
  - **Recovered**: No longer infectious and assumed immune.

- **Transmission** occurs through interactions between *connected agents* (i.e., agents with network edges between them). Importantly, the probability of infection upon contact is **not uniform**. It depends on each agent's **preventative behaviors**, such as:
  - Hygiene (e.g., handwashing, mask usage)
  - Social distancing
  - Vaccination status

These behaviors are encoded as 100-dimensional vectors with values in \([0, 1]\), where higher values indicate stronger protection.

##### **Transmission Behaviors**

The infection decision follows these key steps:

1. **Interaction Safety Evaluation**  
   For each pairwise combination of the agent’s and its neighbor’s *preventative_measures*, a score is computed by multiplying the respective values. These products are summed to produce an aggregate measure of the *absolute safety of interaction*.

2. **Normalization**  
   The summed safety score is normalized by dividing by the square of the length of the preventative measures vector. This results in a *normalized safety score* in the range [0, 1], where higher values indicate stronger mutual preventative behavior.

3. **Infection Rule**  
   If a neighbor is **infected** (`neighbor_state == 2`), the agent has a chance to become infected depending on contagiousness of the pathogen, which is represented by the infection probability `p_infection`, reduced by the *normalized safety score*. Specifically, infection occurs if a sampled random number is less than `p_infection × (1 - normalized safety score)`.

4. **State Update**  
   If the infection condition is met, the agent's state is updated from **susceptible** to **infected**.


# Implementing the `SIRModel`  with SAGESim ##

In the remainder of this notebook, we will build a behavior-aware SIR agent-based model using the `sagesim` library. The following steps mirror the procedure outlined in the project README:

1. **Define the `SIRModel` class**  
   Subclass the `Model` class from `sagesim` to implement the SIR logic, including state transitions and transmission dynamics influenced by agents’ protective behaviors.

2. **Instantiate the model on a contact network**  
   Construct a *Watts–Strogatz* small-world network to represent social interactions. Each node corresponds to an agent, and edges define potential transmission pathways. We initialize the `SIRModel` with this network and a population of behaviorally heterogeneous agents.

3. **Run the simulation and analyze results**  
   Simulate disease spread over time and analyze how individual behaviors shape the epidemic curve and overall dynamics.


## 1. **Define the `SIRModel` Class**

In the SIR example, a susceptible agent checks whether any of its infected neighbors has transmitted the infection. If so, it updates only its own state (e.g., from susceptible to infected). Thus, a reduce function is not needed.

To build the `SIRModel` class, we will

* Defines the space in which agents interact, using a `sagesim.NetworkSpace`.
* Registers the agent breed. For this model, a single breed represents the general population, implemented as the `SIRBreed` class defined later.
* Registers global properties; in this case, `p_infection` (the probability of infection).
* Implements the `create_agent()` method to add agents to the simulation.
* Implements the `connect_agent()` method to define neighbor relationships between agents, which determine possible infection pathways.


In [1]:
!pip uninstall -y sagesim
!pip install ../../.

Found existing installation: sagesim 0.2.0
Uninstalling sagesim-0.2.0:
  Successfully uninstalled sagesim-0.2.0
Processing /home/co1/sagesim_github/sagesim_sfr/SAGESim
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sagesim: filename=sagesim-0.2.0-py3-none-any.whl size=126041 sha256=9c9702b89edcb9b8c70daeac5e5ed421324b9b2b3fc4b630bc4686ede1ad1ef6
  Stored in directory: /tmp/pip-ephem-wheel-cache-v9r4i5ut/wheels/cf/7b/0b/f8e1eaa2e3d2d4372221a438329838cb8fb1d114462c49a9da
Successfully built sagesim


In [2]:
from sagesim.model import Model
from sagesim.space import NetworkSpace # hopefully, we can avoid this import


class SIRModel(Model):
    """
    SIRModel class for the SIR model.
    Inherits from the Model class in the sagesim library.
    """

    def __init__(self, p_infection=0.2, p_recovery=0.2) -> None:
        space = NetworkSpace()
        super().__init__(space)
        self._sir_breed = SIRBreed()

        # Register the breed
        self.register_breed(breed=self._sir_breed)

        # register user-defined global properties
        self.register_global_property("p_infection", p_infection)        
        self.register_global_property("p_recovery", p_recovery)

    # create_agent method takes user-defined properties, that is, the state and preventative_measures, to create an agent
    def create_agent(self, state, preventative_measures):
        agent_id = self.create_agent_of_breed(
            self._sir_breed, state=state, preventative_measures=preventative_measures
        )
        self.get_space().add_agent(agent_id)
        return agent_id

    def connect_agents(self, agent_0, agent_1):
        self.get_space().connect_agents(agent_0, agent_1)


#### **The `SIRBreed` Class**

Now, let's implement the custom agent breed by subclassing the `Breed` class from the `sagesim` library. As mentioned, we need to register agent-specific properties and define and register step functions.

* Each agent in this breed has two primary properties:

  * **`state`**: A categorical variable indicating the agent's infection status. The three SIR states are encoded as:

    * `1` — Susceptible
    * `2` — Infected
    * `3` — Recovered

  * **`preventative_measures`**: A list of 100 floating-point values that represent the agent’s behavioral traits—such as hygiene practices, social distancing adherence, or vaccination status—that affect their likelihood of becoming infected.


In [3]:
from enum import Enum
from random import random

from sagesim.breed import Breed
from sir_step_func import step_func

# Define the SIRState enumeration for agent states
class SIRState(Enum):
    SUSCEPTIBLE = 1
    INFECTED = 2
    RECOVERED = 3

class SIRBreed(Breed):
    """
    SIRBreed class the SIR model.
    Inherits from the Breed class in the sagesim library.
    """

    def __init__(self) -> None:
        name = "SIR"
        super().__init__(name) 
        # Register properties for the breed
        self.register_property("state", SIRState.SUSCEPTIBLE.value) 
        self.register_property("preventative_measures", [random() for _ in range(100)])
        # Register the step function
        self.register_step_func(step_func, "sir_step_func.py", 0)


/home/co1/anaconda3/envs/sagesim/lib/python3.12/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


* Next, we define the `step_func`. For the SIR model, we use a single step function that governs how an agent’s state evolves over time—for example, determining whether a susceptible agent becomes infected or an infected agent recovers. We implement this step function based on the transmission dynamics discussed earlier.
This step function should be implemented as a callback, which accepts the following arguments:
    - `agent_ids`: 1D `cupy.array` of ids of agents this rank is assigned followed by the agents neighboring those agents
    - `agent_index`: An `int` the index in the property tensors occupied by this agent.
    - `globals`: a 1D `cupy.array` of global properties. The first element will the simulation tick, followed by user defined global properties in order they were registered.
    - `breeds`: 1D `cupy.array` of floats indicating the index of the breed each agent belongs to, ordered in the order the breeds were registered to the `sagesim.model`.
    - `locations`: 2+D `cupy.array` of floats containing location information of each agent. In the case of `NetworkSpace` the 2nd dim of this array will contain the neighbors of each agent, padded by `cp.nan`.
    - A 1+D `cupy.array` property tensor for each breed property registered for all breeds, in order the breeds were registered to the model and that the properties were registered to each breed. The dimensions for each property tensor will be equal to the dimensions of the property when the agent was created or the property was registered to the breed plus 1. 

Changes made to the agent must be recorded by editing the respective index of one of the property tensor for the agent. The step function will be executed per agent by an independent GPU thread.

In [4]:
from time import time
from random import random, sample

import networkx as nx
from mpi4py import MPI
import cupy as cp
from cupyx import jit

from sagesim.utils import get_this_agent_data_from_tensor, set_this_agent_data_from_tensor, get_neighbor_data_from_tensor


# Define the step function to be registered for SIRBreed
@jit.rawkernel(device="cuda")
def step_func(agent_index, globals, agent_ids, breeds, locations, state_tensor, preventative_measures_tensor):
    """
    At each simulation step, this function evaluates a subset of agents—either all agents in a serial run or a partition assigned to
    a specific rank in parallel processing—and determines whether an agent's state should change based on interactions with its neighbors
    and their respective preventative behaviors.

    Parameters:
    ----------
    agent_index : int
        Index of the agent being evaluated in the agent_ids list. 
    globals : cupy.array
        Global parameters; 
        the zero-th global parameter is by default the simulation tick, 
        the first item will be our infection probability.
        the second item will be our recovery probability.
    agent_ids : cupy.array
        The tensor that contains the IDs of all agents assigned to the current rank, and their neighbors.
    breeds : cupy.array
        1D cupy.array of breed objects (unused here as we only have one type of breed, but must passed for interface compatibility).
    locations : cupy.array
        Adjacency list specifying neighbors for each agent, padded with cp.nan.
    state_tensor : cupy.array
        1D cupy array of current state of each agent.
    preventative_measures_tensor : cupy.array[]
        2D cupy array of vectors representing each agent’s preventative behaviors. 
    Returns:
    -------
    None
        The function updates the `states` list in-place if an agent becomes infected.
    """
    # Get the list of neighboring agent IDs for the current agent based on network topology
    neighbor_ids = locations[agent_index]

    # Draw a random float in [0, 1) for stochastic decision-making
    rand = random()  # can replace with step_func_helper_get_random_float(rng_states, id)

    # Retrieve the global infection and recovery probabilities defined in the model
    p_infection, p_recovery = globals[1:]

    # Get the current state of the agent (e.g., susceptible, infected, recovered)
    agent_state = int(get_this_agent_data_from_tensor(agent_index, state_tensor))
    
    # Get the preventative measures vector for the current agent
    agent_preventative_measures = get_this_agent_data_from_tensor(agent_index, preventative_measures_tensor)

    # If agent is infected and the recovery condition passes, update agent’s state
    if agent_state == 2 and rand < p_recovery:
        set_this_agent_data_from_tensor(agent_index, state_tensor, 3)  # Agent becomes infected
    elif agent_state == 1:
        # Loop through each neighbor ID
        i = 0
        while i < len(neighbor_ids) and not cp.isnan(neighbor_ids[i]):
            neighbor_id = neighbor_ids[i]
            
            # Retrieve the state of the neighbor (e.g., susceptible, infected, recovered)
            neighbor_state = int(get_neighbor_data_from_tensor(agent_ids, neighbor_id, state_tensor))

            # Get the preventative measures vector of the neighbor
            neighbor_preventative_measures = get_neighbor_data_from_tensor(agent_ids, neighbor_id, preventative_measures_tensor)

            # Initialize cumulative safety score for the interaction
            abs_safety_of_interaction = 0.0

            # Calculate total safety of interaction based on pairwise product of measures
            for n in range(len(agent_preventative_measures)):
                for m in range(len(neighbor_preventative_measures)):
                    abs_safety_of_interaction += (
                        agent_preventative_measures[n] * neighbor_preventative_measures[m]
                    )

            # Normalize the safety score to be in [0, 1]
            normalized_safety_of_interaction = abs_safety_of_interaction / (
                len(agent_preventative_measures) ** 2
            )

            # If neighbor is infected and the infection condition passes, update agent’s state
            if neighbor_state == 2 and rand < p_infection * (
                1 - normalized_safety_of_interaction
            ):
                set_this_agent_data_from_tensor(agent_index, state_tensor, 2)  # Agent becomes infected
            i += 1


Due to limitations with defining GPU kernels, the step functions must be defined in a importable Python script file. For convenience, we have defined the step function in `sir_step_func.py`.

## **Instantiate the SIR Model**

1. **Generate the Contact Network**  
   Create a *Watts–Strogatz* small-world graph to approximate real-world contact patterns. This topology offers high clustering and short average path lengths, making it ideal for modeling disease transmission. Each node represents an agent, and each edge represents a potential transmission link.

2. **Create and Add Agents**  
   Instantiate the `SIRModel`, then call `create_agent()` for each node to initialize agents in the **susceptible** state with their corresponding `preventative_measures` vector.

3. **Connect Agents**  
   Use the model’s `connect_agents(agent_a, agent_b)` method to add edges between agents according to the network structure.

4. **Initialize Infections**  
   Randomly select `num_infected` agents and set their state to **infected** to seed the simulation.


In [5]:
num_agents = 1000
num_init_connections = 20
rewiring_prob = 0.1

num_infected = 10

# Generate the Contact Network
network = nx.watts_strogatz_graph(num_agents, num_init_connections, rewiring_prob, seed=42)

# Instantiate the SIR Model
model = SIRModel()
model.setup(use_gpu=True)  # Enables GPU acceleration if available

# Create agents 
for n in network.nodes:
    preventative_measures = [random() for _ in range(100)] 
    model.create_agent(SIRState.SUSCEPTIBLE.value, preventative_measures)

# Connect agents in the network
for edge in network.edges:
    model.connect_agents(edge[0], edge[1])

# Infect a random sample of agents  
for n in sample(sorted(network.nodes), num_infected):
    model.set_agent_property_value(n, "state", SIRState.INFECTED.value)


## **Running the Simulation and Analyzing Results**

To execute the simulation, call `model.simulate(ticks=10, sync_workers_every_n_ticks=1)` to simulate 10 time steps, synchronizing all workers after each tick.

1. Single-CPU/GPU, Single-Rank Execution
Begin by running the simulation with a single MPI rank directly in this notebook. This approach is useful for validating correctness and establishing a performance baseline. You may also time the execution to measure baseline performance.

2. Single-CPU/GPU, Multi-Rank Execution
To assess the benefits of parallel execution, run the simulation with multiple MPI ranks. Although this is not supported within the notebook interface, you can execute the same code from a script using the command: `mpiexec -n 4 python tutorial_run.py`

3. Multi-GPU, Multi-Rank Execution on HPC
For large-scale simulations and faster runtimes, deploy the model in a high-performance computing (HPC) environment using multiple GPUs. Refer to the final section of `sagesim_tutorial.md` -- **High-Performance Computing (HPC) with `sagesim`** -- for instructions on writing a submission script and launching `tutorial_run.py` via a job scheduler.

In [6]:
# 1. Single-CPU/GPU, Single-Rank Execution

# MPI environment setup
comm = MPI.COMM_WORLD
num_workers = comm.Get_size()
worker = comm.Get_rank()

# Run the simulation with 1 rank, and measure the time taken
simulate_start = time()
model.simulate(ticks = 10, sync_workers_every_n_ticks=1)
simulate_end = time()
simulate_duration = simulate_end - simulate_start
print(f"Simulation with 1 rank took {simulate_duration:.2f} seconds.")


Simulation with 1 rank took 3.71 seconds.


In [7]:
# Get the state of each agents after 10 simulation runs
result = [
    SIRState(model.get_agent_property_value(agent_id, property_name="state"))
    for agent_id in range(num_agents)
    if model.get_agent_property_value(agent_id, property_name="state") is not None
]

# count the number of infected agents
num_infected = sum(1 for state in result if state == SIRState.INFECTED)
num_recovered = sum(1 for state in result if state == SIRState.RECOVERED)
num_susceptible = sum(1 for state in result if state == SIRState.SUSCEPTIBLE)
print(f"Number of infected agents: {num_infected}")
print(f"Number of recovered agents: {num_recovered}")
print(f"Number of susceptible agents: {num_susceptible}")

Number of infected agents: 10
Number of recovered agents: 0
Number of susceptible agents: 990


In [10]:
# Single-CPU/GPU, Multi-Rank Execution
# run the simulation with 4 ranks   
!mpirun -n 4 python tutorial_run.py

/home/co1/anaconda3/envs/sagesim/lib/python3.12/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')
/home/co1/anaconda3/envs/sagesim/lib/python3.12/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')
/home/co1/anaconda3/envs/sagesim/lib/python3.12/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')
/home/co1/anaconda3/envs/sagesim/lib/python3.12/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')
Simulation took 4.15 seconds.
Number of infected agents: 15
Number of re